<a href="https://colab.research.google.com/github/soaminfosec/MSDefnder_Pull_request/blob/main/Inventorycompare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# ✅ Step 1: Install dependencies
!pip install pandas openpyxl

# ✅ Step 2: Import modules
from google.colab import files
import pandas as pd
import io

# ✅ Step 3: Upload & read CSV/XLSX
def upload_and_read(prompt):
    print(f"⬆️ Please upload {prompt} (.csv or .xlsx):")
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]

    if filename.endswith(".csv"):
        return pd.read_csv(io.BytesIO(uploaded[filename]))
    elif filename.endswith(".xlsx"):
        return pd.read_excel(io.BytesIO(uploaded[filename]))
    else:
        raise ValueError("❌ Unsupported file format. Upload .csv or .xlsx only.")

# ✅ Step 4: Comparison Logic
def compare_inventory(inventory_df, apex_df, ds_df=None):
    results = []

    for _, inv_row in inventory_df.iterrows():
        inv_name = str(inv_row['COMPUTER NAME']).strip().lower()
        inv_ip = str(inv_row['IP']).strip()
        inv_status = inv_row.get('Status', '')

        # --- Apex One match ---
        apex_match_name = apex_df[apex_df['Endpoint'].str.lower() == inv_name]
        apex_match_ip = apex_df[apex_df['IP Address'].astype(str).str.strip() == inv_ip]

        apex_row = None
        apex_status = "Not reporting to Apex one for both IP and hostname"
        apex_endpoint = apex_ip = ""

        if not apex_match_name.empty:
            apex_row = apex_match_name.iloc[0]
            apex_endpoint = apex_row['Endpoint']
            apex_ip = apex_row['IP Address']
            apex_status = "Matched with hostname"
        elif not apex_match_ip.empty:
            apex_row = apex_match_ip.iloc[0]
            apex_endpoint = apex_row['Endpoint']
            apex_ip = apex_row['IP Address']
            apex_status = "Matched with IP address"

        # --- Deep Security match ---
        ds_status = ds_name = "Not reported to DS"
        if ds_df is not None:
            ds_match_name = ds_df[ds_df['Name'].str.lower() == inv_name]
            if not ds_match_name.empty:
                ds_row = ds_match_name.iloc[0]
                ds_name = ds_row['Name']
                ds_status = "Reported to DS"

        # --- Final Status ---
        if apex_status.startswith("Matched") and "Reported to DS" in ds_status:
            combined_status = f"Machine is reporting to both DS and APEX one console ({apex_status})."
        elif apex_status.startswith("Matched") and "Not reported" in ds_status:
            combined_status = "Machine is reporting to Apex One but not DS."
        elif "Not reporting" in apex_status and "Reported to DS" in ds_status:
            combined_status = "Machine is reporting to DS not Apex one."
        else:
            combined_status = "Machine is not reporting to either console."

        results.append({
            'COMPUTER NAME': inv_name,
            'IP': inv_ip,
            'Status': inv_status,
            'Comparing Endpoint': apex_endpoint,
            'IP Address': apex_ip,
            'Apex One Status': apex_status,
            'Deep Security Name': ds_name,
            'DS and Apex One Status': combined_status
        })

    return pd.DataFrame(results)

# ✅ Step 5: Upload files
inventory_df = upload_and_read("Inventory file")
apex_df = upload_and_read("Apex One file")

# Optional DS
upload_ds = input("Do you have a Deep Security file to upload? (y/n): ").strip().lower()
ds_df = None
if upload_ds == 'y':
    ds_df = upload_and_read("Deep Security file")

# ✅ Step 6: Normalize columns
inventory_df.columns = [c.strip() for c in inventory_df.columns]
apex_df.columns = [c.strip() for c in apex_df.columns]
if ds_df is not None:
    ds_df.columns = [c.strip() for c in ds_df.columns]

# ✅ Step 7: Compare and generate result
result_df = compare_inventory(inventory_df, apex_df, ds_df)

# ✅ Step 8: Download the result
result_filename = "comparison_result.csv"
result_df.to_csv(result_filename, index=False)
print(f"✅ Result saved as {result_filename}")
files.download(result_filename)


⬆️ Please upload Inventory file (.csv or .xlsx):


Saving IOCL_Inventory.xlsx to IOCL_Inventory.xlsx
⬆️ Please upload Apex One file (.csv or .xlsx):


Saving IOCL_DR1.xlsx to IOCL_DR1.xlsx
Do you have a Deep Security file to upload? (y/n): n
✅ Result saved as comparison_result.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>